In [9]:
import numpy as np


In [10]:
train_file=open("Data/RU/train","r")
train_words=[""]
tags=["START"]
for l in train_file:
    if l!="\n":
        lst=l.split()
        x=""
        for i in range(len(lst)-1):
            x+=lst[i]+" "
        x=x[0:-1]
        y=lst[-1]
        train_words.append(x)
        tags.append(y)
tags.append("STOP")
train_words.append("")
train_words=np.array(train_words)
tags=np.array(tags)

In [3]:
test_file=open("Data/RU/dev.in","r")
test_words=[]
for l in test_file:
    if l!="\n":
        test_words.append(l[0:-1])
test_words=np.array(test_words)
print(test_words)

['Интерьер' ',' 'интерьер' ... 'даже' 'скромное' '.']


In [4]:
def transition_helper(u,v,tags):
    count_u=0
    count_u_to_v=0
    for i in range(len(tags)):
        if tags[i]==u:
            count_u+=1
            if tags[i+1]==v:
                count_u_to_v+=1
    return count_u_to_v/count_u

def transition(tags):
    transition_parameters={}
    for i in range(len(tags)):
        if tags[i]=="STOP":
            break
        if (tags[i],tags[i+1]) not in transition_parameters:
            transition_parameters[(tags[i],tags[i+1])]=transition_helper(tags[i],tags[i+1],tags)
    return transition_parameters

In [5]:
def emission_helper(x, y,tags, words=None, k=1):
    if x=="#UNK":
        return k/(np.sum(tags == y)+k)
    count_y_to_x, count_y = 0, 0
    for i in range(len(tags)):
        if tags[i] == y:
            count_y += 1
            if words[i] == x:
                count_y_to_x += 1
    return (count_y_to_x) / (count_y + k)

def emission(tags,train_words,test_words):
    emission_word_tag={}
    for x in np.unique(test_words):
        y_val={}
        for y in np.unique(tags):
            if x in train_words:
                ep = emission_helper(x, y,tags,train_words)
                y_val.update({y:ep})
            else:
                ep = emission_helper("#UNK#", y,tags,train_words)
                y_val.update({y:ep})
        emission_word_tag[x] = y_val
    return emission_word_tag

In [6]:
emission_word_tag=emission(tags,train_words,test_words)
transition_parameters=transition(tags)

print(emission_word_tag)

{'!': {'B-negative': 0.04044943820224719, 'B-neutral': 0.0673076923076923, 'B-positive': 0.029601722282023683, 'I-negative': 0.0, 'I-neutral': 0.0, 'I-positive': 0.0, 'O': 0.019739926468773903, 'START': 0.0, 'STOP': 0.5}, '"': {'B-negative': 0.0044943820224719105, 'B-neutral': 0.004807692307692308, 'B-positive': 0.002691065662002153, 'I-negative': 0.007042253521126761, 'I-neutral': 0.057971014492753624, 'I-positive': 0.026755852842809364, 'O': 0.004860956892935574, 'START': 0.0, 'STOP': 0.0}, '%': {'B-negative': 0.0, 'B-neutral': 0.0, 'B-positive': 0.0005382131324004305, 'I-negative': 0.0, 'I-neutral': 0.0, 'I-positive': 0.0, 'O': 0.00027142398894564116, 'START': 0.0, 'STOP': 0.0}, '(': {'B-negative': 0.011235955056179775, 'B-neutral': 0.004807692307692308, 'B-positive': 0.0016146393972012918, 'I-negative': 0.0, 'I-neutral': 0.0, 'I-positive': 0.0, 'O': 0.006464825918523454, 'START': 0.0, 'STOP': 0.0}, ')': {'B-negative': 0.006741573033707865, 'B-neutral': 0.009615384615384616, 'B-posi

In [8]:
for i in emission_word_tag:
    if emission_word_tag[i]["STOP"]!=0 or emission_word_tag[i]["START"]!=0:
        print(i,emission_word_tag[i]["START"],emission_word_tag[i]["STOP"])

! 0.0 0.5


In [ ]:
def viterbi(sequence, tags, transition_parameters, emission_parameters):
    n = len(sequence)
    num_tags = len(tags)

    # Initialize the Viterbi matrices and backpointers
    left_to_right = np.zeros((num_tags, n))
    right_to_left = np.zeros((num_tags, n))

    # Initialization step
    for i in range(num_tags):
        if ("START", tags[i]) not in transition_parameters:
            left_to_right[i, 0] = 0 * emission_parameters[sequence[0]][tags[i]]
        else:
            left_to_right[i, 0] = transition_parameters[("START", tags[i])] * emission_parameters[sequence[0]][tags[i]]

    # Viterbi loop
    for t in range(1, n):
        for j in range(num_tags):
            max_prob = -1
            max_backpointer = -1

            for i in range(num_tags):
                if (tags[i],tags[j]) not in transition_parameters:
                    prob = left_to_right[i, t - 1] * 0 * emission_parameters[sequence[t]][tags[j]]
                else:
                    prob = left_to_right[i, t - 1] * transition_parameters[(tags[i], tags[j])] * emission_parameters[sequence[t]][tags[j]]

                if prob > max_prob:
                    max_prob = prob
                    max_backpointer = i

            left_to_right[j, t] = max_prob
            right_to_left[j, t] = max_backpointer

    # Termination step
    max_prob = -1
    max_backpointer = -1
    for i in range(num_tags):
        if (tags[i],"STOP") not in transition_parameters:
            prob = left_to_right[i, n - 1] * 0
        else:    
            prob = left_to_right[i, n - 1] * transition_parameters[(tags[i], "STOP")]

        if prob > max_prob:
            max_prob = prob
            max_backpointer = i

    left_to_right[num_tags-1, n-1] = max_prob
    right_to_left[num_tags-1, n-1] = max_backpointer

    # Retrieve the best path using backpointers
    best_path = ["STOP"]
    for t in range(n-1, 0, -1):
        best_path.insert(0, tags[right_to_left[tags.index(best_path[0]), t]])

    return best_path



In [ ]:
pred_output=open("DATA/RU/dev.p2.out","w")
sequence=[]
for l in test_file:
    if l!="\n":
        sequence.append(l[0:-1])
    else:
        predicted_tags=viterbi(sequence,tags,transition_parameters,emission_word_tag)
        for i in range(1,len(predicted_tags)-1):
            pred_output.write(sequence[i]+" "+predicted_tags[i]+"\n")
        pred_output.write("\n")